# 深度卷积生成对抗网络（DCGAN）实现

## 理论基础

生成对抗网络（GAN）由Ian Goodfellow于2014年提出，包含两个相互竞争的神经网络：

1. **生成器（Generator）**：从潜在空间的随机噪声生成逼真的样本
2. **判别器（Discriminator）**：判断输入样本是真实样本还是生成样本

### 数学原理

GAN的训练目标是最小化生成器的损失，同时最大化判别器的损失，形成一个极小极大博弈：

$$\min_G \max_D V(D,G) = \mathbb{E}_{x \sim p_{data}(x)}[\log D(x)] + \mathbb{E}_{z \sim p_z(z)}[\log(1-D(G(z)))]$$

其中：
- $D(x)$：判别器对真实样本的输出概率
- $G(z)$：生成器从噪声$z$生成的样本
- $p_{data}$：真实数据分布
- $p_z$：潜在空间噪声分布（通常为高斯分布）

### DCGAN架构特点

DCGAN（Deep Convolutional GAN）是GAN的改进版本，主要特点：
- 使用卷积和转置卷积替代全连接层
- 取消池化层，使用步进卷积进行上/下采样
- 使用Batch Normalization（除生成器输出层和判别器输入层外）
- 生成器使用ReLU激活（输出层使用tanh）
- 判别器使用LeakyReLU激活

In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

# 设置随机种子以确保结果可复现
RANDOM_SEED = 42
np.random.seed(RANDOM_SEED)
tf.random.set_seed(RANDOM_SEED)

# ============================================================================
# 超参数配置
# ============================================================================
latent_dim = 32  # 潜在空间维度：控制生成器输入的噪声向量维度
height, width, channels = 32, 32, 3  # 生成图像尺寸（与CIFAR-10一致）

# ============================================================================
# 生成器网络架构
# ============================================================================
# 生成器的作用：将低维潜在空间的随机噪声映射到高维图像空间
# 架构设计遵循DCGAN原则：全卷积网络，使用LeakyReLU和转置卷积

generator_input = keras.Input(shape=(latent_dim,), name='generator_input')

# 第一层：全连接层扩展维度
# 将32维噪声向量扩展到16x16x128的特征图
x = layers.Dense(128 * 16 * 16, name='dense_expand')(generator_input)
x = layers.LeakyReLU(negative_slope=0.2, name='leaky_relu_1')(x)
x = layers.Reshape((16, 16, 128), name='reshape')(x)

# 第二层：卷积层特征提取
# 保持空间尺寸不变，增加特征通道数
x = layers.Conv2D(256, kernel_size=5, padding='same', name='conv2d_1')(x)
x = layers.LeakyReLU(negative_slope=0.2, name='leaky_relu_2')(x)

# 第三层：转置卷积上采样
# 将16x16上采样到32x32（步长为2）
x = layers.Conv2DTranspose(256, kernel_size=4, strides=2, padding='same', name='conv2d_transpose')(x)
x = layers.LeakyReLU(negative_slope=0.2, name='leaky_relu_3')(x)

# 第四、五层：卷积层深度特征提取
# 保持32x32尺寸，进一步提炼特征
x = layers.Conv2D(256, kernel_size=5, padding='same', name='conv2d_2')(x)
x = layers.LeakyReLU(negative_slope=0.2, name='leaky_relu_4')(x)
x = layers.Conv2D(256, kernel_size=5, padding='same', name='conv2d_3')(x)
x = layers.LeakyReLU(negative_slope=0.2, name='leaky_relu_5')(x)

# 输出层：生成RGB图像
# 使用tanh激活函数将输出限制在[-1, 1]范围（与数据归一化对应）
x = layers.Conv2D(channels, kernel_size=7, activation='tanh', padding='same', name='output')(x)

# 实例化生成器模型
generator = keras.models.Model(generator_input, x, name='generator')
generator.summary()

# 判别器网络架构

判别器是一个二分类器，用于区分真实图像和生成图像。

**设计原则：**
- 使用步进卷积进行下采样（替代池化层）
- 使用LeakyReLU激活函数（避免梯度消失）
- 添加Dropout防止过拟合
- 输出单个概率值（sigmoid激活）

In [ ]:
# ============================================================================
# 判别器网络架构
# ============================================================================
# 判别器的作用：将高维图像空间映射到[0,1]的概率值
# 输出接近1表示真实图像，接近0表示生成图像

discriminator_input = layers.Input(shape=(height, width, channels), name='discriminator_input')

# 第一层：初始卷积特征提取
# 32x32x3 -> 30x30x128
x = layers.Conv2D(128, kernel_size=3, name='disc_conv2d_1')(discriminator_input)
x = layers.LeakyReLU(negative_slope=0.2, name='disc_leaky_relu_1')(x)

# 第二层：步进卷积下采样
# 30x30x128 -> 14x14x128
x = layers.Conv2D(128, kernel_size=4, strides=2, name='disc_conv2d_2')(x)
x = layers.LeakyReLU(negative_slope=0.2, name='disc_leaky_relu_2')(x)

# 第三层：步进卷积下采样
# 14x14x128 -> 6x6x128
x = layers.Conv2D(128, kernel_size=4, strides=2, name='disc_conv2d_3')(x)
x = layers.LeakyReLU(negative_slope=0.2, name='disc_leaky_relu_3')(x)

# 第四层：步进卷积下采样
# 6x6x128 -> 2x2x128
x = layers.Conv2D(128, kernel_size=4, strides=2, name='disc_conv2d_4')(x)
x = layers.LeakyReLU(negative_slope=0.2, name='disc_leaky_relu_4')(x)

# 展平特征
x = layers.Flatten(name='flatten')(x)

# Dropout层：随机丢弃30%的神经元，防止判别器过拟合
# 过强的判别器会导致生成器难以学习
x = layers.Dropout(0.3, name='dropout')(x)

# 输出层：二分类概率
# sigmoid输出范围[0,1]，表示输入为真实图像的概率
x = layers.Dense(1, activation='sigmoid', name='disc_output')(x)

# 实例化判别器模型
discriminator = keras.models.Model(discriminator_input, x, name='discriminator')
discriminator.summary()

# ============================================================================
# 判别器优化器配置
# ============================================================================
# RMSprop优化器：适合处理非平稳目标（GAN训练的特点）
# - learning_rate: 学习率，控制参数更新步长
# - clipvalue: 梯度裁剪，防止梯度爆炸

discriminator_optimizer = keras.optimizers.RMSprop(
    learning_rate=0.0008,
    clipvalue=1.0
)

# 编译判别器
# 使用二元交叉熵损失，因为这是一个二分类问题
discriminator.compile(
    optimizer=discriminator_optimizer,
    loss='binary_crossentropy',
    metrics=['accuracy']
)

# 构建对抗网络（GAN）

对抗网络将生成器和判别器串联，用于训练生成器。

**训练策略：**
1. 固定判别器权重（`trainable=False`）
2. 生成器生成假样本
3. 判别器评估假样本
4. 通过反向传播更新生成器权重
5. 目标：让判别器将生成样本误判为真实样本

**交替训练：**
- 训练判别器时：使用真实和生成样本，更新判别器权重
- 训练生成器时：通过GAN模型，固定判别器权重，只更新生成器权重

In [ ]:
# ============================================================================
# 构建对抗网络（GAN）
# ============================================================================
# GAN模型用于训练生成器：
# 输入：潜在空间噪声 -> 生成器 -> 生成图像 -> 判别器 -> 真假概率
# 训练时判别器权重被冻结，只更新生成器权重

# 冻结判别器权重
# 这样在训练GAN时，只有生成器的权重会被更新
discriminator.trainable = False

# 构建GAN模型
gan_input = keras.Input(shape=(latent_dim,), name='gan_input')
gan_output = discriminator(generator(gan_input))
gan = keras.models.Model(gan_input, gan_output, name='gan')

# ============================================================================
# GAN优化器配置
# ============================================================================
# 生成器的学习率设置为判别器的一半
# 这样可以避免生成器学习过快导致训练不稳定

gan_optimizer = keras.optimizers.RMSprop(
    learning_rate=0.0004,
    clipvalue=1.0
)

# 编译GAN模型
# 生成器的目标：让判别器输出接近1（即将生成图像误判为真实图像）
gan.compile(
    optimizer=gan_optimizer,
    loss='binary_crossentropy'
)

print('GAN模型构建完成')
print(f'生成器参数量: {generator.count_params():,}')
print(f'判别器参数量: {discriminator.count_params():,}')
print(f'GAN总参数量: {gan.count_params():,}')

# 训练DCGAN

## 训练流程

GAN的训练是一个交替优化的过程：

### 1. 训练判别器
- 从数据集采样真实图像，标签为1
- 生成器生成假图像，标签为0
- 将真假图像混合，训练判别器区分它们

### 2. 训练生成器
- 生成假图像
- 使用标签1（欺骗判别器）
- 通过冻结的判别器反向传播，只更新生成器权重

## 训练技巧

1. **标签平滑**：给标签添加小噪声，防止判别器过于自信
2. **标签翻转**：偶尔翻转标签，增加训练稳定性
3. **学习率平衡**：生成器学习率通常低于判别器
4. **梯度裁剪**：防止梯度爆炸
5. **监控损失**：判别器损失趋近0表示过拟合，需要调整

## 数据预处理

CIFAR-10数据集包含60000张32x32的彩色图像，分为10类。
GAN训练不需要标签，只使用图像数据。
图像归一化到[-1, 1]范围，与生成器的tanh输出对应。

In [ ]:
# ============================================================================
# 数据加载与预处理
# ============================================================================

# 加载CIFAR-10数据集
print('正在加载CIFAR-10数据集...')
(x_train, y_train), (_, _) = tf.keras.datasets.cifar10.load_data()

# 数据预处理
# 1. 归一化到[-1, 1]范围（对应生成器tanh输出）
# 2. 不需要标签，因为GAN是无监督学习
x_train = x_train.astype('float32')
x_train = (x_train - 127.5) / 127.5  # 归一化到[-1, 1]

print(f'训练集形状: {x_train.shape}')
print(f'数据范围: [{x_train.min():.2f}, {x_train.max():.2f}]')

# ============================================================================
# 训练配置
# ============================================================================

# 训练超参数
iterations = 10000  # 总迭代次数
batch_size = 20  # 批次大小
save_interval = 100  # 保存间隔

# 创建输出目录
output_dir = './gan_output'
os.makedirs(output_dir, exist_ok=True)
print(f'输出目录: {output_dir}')

# ============================================================================
# 可视化辅助函数
# ============================================================================

def save_generated_images(epoch, generator, latent_dim, examples=10, dim=(1, 10), figsize=(20, 2)):
    """
    生成并保存图像网格
    
    参数:
        epoch: 当前迭代次数
        generator: 生成器模型
        latent_dim: 潜在空间维度
        examples: 生成图像数量
        dim: 网格维度(行, 列)
        figsize: 图像尺寸
    """
    noise = np.random.normal(0, 1, size=(examples, latent_dim))
    generated_images = generator.predict(noise, verbose=0)
    
    # 反归一化到[0, 1]用于显示
    generated_images = (generated_images + 1) / 2.0
    generated_images = np.clip(generated_images, 0, 1)
    
    plt.figure(figsize=figsize)
    for i in range(examples):
        plt.subplot(dim[0], dim[1], i + 1)
        plt.imshow(generated_images[i])
        plt.axis('off')
    
    plt.tight_layout()
    plt.savefig(f'{output_dir}/generated_epoch_{epoch}.png', dpi=100, bbox_inches='tight')
    plt.close()

# ============================================================================
# 训练循环
# ============================================================================

print('\n开始训练DCGAN...')
print(f'总迭代次数: {iterations}')
print(f'批次大小: {batch_size}')
print('-' * 80)

# 训练历史记录
d_losses = []
g_losses = []
d_accuracies = []

start_idx = 0

for step in range(iterations):
    # ------------------------------------------------------------------------
    # 阶段1: 训练判别器
    # ------------------------------------------------------------------------
    
    # 解冻判别器权重
    discriminator.trainable = True
    
    # 1.1 采样真实图像
    stop_idx = start_idx + batch_size
    real_images = x_train[start_idx:stop_idx]
    
    # 1.2 生成假图像
    random_latent_vectors = np.random.normal(size=(batch_size, latent_dim))
    generated_images = generator.predict(random_latent_vectors, verbose=0)
    
    # 1.3 合并真假图像
    combined_images = np.concatenate([real_images, generated_images])
    
    # 1.4 创建标签
    # 真实图像标签为1，生成图像标签为0
    # 添加标签平滑：给标签加入小噪声，防止判别器过于自信
    labels = np.concatenate([
        np.ones((batch_size, 1)),
        np.zeros((batch_size, 1))
    ])
    labels += 0.05 * np.random.random(labels.shape)
    
    # 1.5 训练判别器
    d_loss, d_acc = discriminator.train_on_batch(combined_images, labels)
    
    # ------------------------------------------------------------------------
    # 阶段2: 训练生成器
    # ------------------------------------------------------------------------
    
    # 冻结判别器权重
    discriminator.trainable = False
    
    # 2.1 采样潜在空间噪声
    random_latent_vectors = np.random.normal(size=(batch_size, latent_dim))
    
    # 2.2 创建欺骗性标签
    # 目标是让判别器认为生成的图像是真实的（标签为1）
    misleading_targets = np.ones((batch_size, 1))
    
    # 2.3 训练生成器（通过GAN模型）
    g_loss = gan.train_on_batch(random_latent_vectors, misleading_targets)
    
    # ------------------------------------------------------------------------
    # 更新训练状态
    # ------------------------------------------------------------------------
    
    # 记录损失
    d_losses.append(d_loss)
    g_losses.append(g_loss)
    d_accuracies.append(d_acc)
    
    # 更新数据索引
    start_idx += batch_size
    if start_idx > len(x_train) - batch_size:
        start_idx = 0
    
    # ------------------------------------------------------------------------
    # 定期保存和可视化
    # ------------------------------------------------------------------------
    
    if step % save_interval == 0:
        # 打印训练进度
        print(f'Step {step:5d}/{iterations} | '
              f'D Loss: {d_loss:.4f} | '
              f'G Loss: {g_loss:.4f} | '
              f'D Acc: {d_acc:.4f}')
        
        # 保存生成的图像
        save_generated_images(step, generator, latent_dim)
        
        # 保存模型权重
        gan.save_weights(f'{output_dir}/gan_weights_step_{step}.h5')
        
        # 判断训练状态
        if d_acc < 0.5:
            print('  ⚠️  警告: 判别器准确率过低，可能需要调整学习率')
        elif d_acc > 0.95:
            print('  ⚠️  警告: 判别器准确率过高，可能过拟合')

print('-' * 80)
print('训练完成！')

# ============================================================================
# 训练结果可视化
# ============================================================================

plt.figure(figsize=(15, 5))

# 绘制损失曲线
plt.subplot(1, 3, 1)
plt.plot(d_losses, label='Discriminator Loss', alpha=0.7)
plt.plot(g_losses, label='Generator Loss', alpha=0.7)
plt.xlabel('Iteration')
plt.ylabel('Loss')
plt.title('Training Loss')
plt.legend()
plt.grid(True, alpha=0.3)

# 绘制判别器准确率
plt.subplot(1, 3, 2)
plt.plot(d_accuracies, label='Discriminator Accuracy', alpha=0.7, color='green')
plt.axhline(y=0.5, color='r', linestyle='--', label='Random Guess')
plt.xlabel('Iteration')
plt.ylabel('Accuracy')
plt.title('Discriminator Accuracy')
plt.legend()
plt.grid(True, alpha=0.3)

# 绘制损失比率
plt.subplot(1, 3, 3)
loss_ratio = np.array(g_losses) / (np.array(d_losses) + 1e-8)
plt.plot(loss_ratio, label='G Loss / D Loss', alpha=0.7, color='purple')
plt.axhline(y=1, color='r', linestyle='--', label='Equal Loss')
plt.xlabel('Iteration')
plt.ylabel('Loss Ratio')
plt.title('Generator/Discriminator Loss Ratio')
plt.legend()
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig(f'{output_dir}/training_history.png', dpi=150, bbox_inches='tight')
plt.show()

print(f'\n所有结果已保存到: {output_dir}')
print(f'- 生成图像: generated_epoch_*.png')
print(f'- 模型权重: gan_weights_step_*.h5')
print(f'- 训练历史: training_history.png')

# 潜在空间探索与插值

训练完成后，我们可以探索潜在空间的特性：
- 潜在空间的连续性：相近的噪声向量生成相似的图像
- 线性插值：在两个噪声向量之间平滑过渡
- 语义方向：潜在空间中的特定方向可能对应特定的语义属性

In [ ]:
# ============================================================================
# 潜在空间插值实验
# ============================================================================

def interpolate_latent_space(generator, latent_dim, n_steps=10):
    """
    在潜在空间中进行线性插值，生成平滑过渡的图像序列
    
    参数:
        generator: 生成器模型
        latent_dim: 潜在空间维度
        n_steps: 插值步数
    """
    # 生成两个随机噪声向量
    z1 = np.random.normal(size=(1, latent_dim))
    z2 = np.random.normal(size=(1, latent_dim))
    
    # 线性插值
    alphas = np.linspace(0, 1, n_steps)
    interpolated_vectors = np.array([alpha * z1 + (1 - alpha) * z2 for alpha in alphas])
    interpolated_vectors = interpolated_vectors.reshape(n_steps, latent_dim)
    
    # 生成图像
    generated_images = generator.predict(interpolated_vectors, verbose=0)
    generated_images = (generated_images + 1) / 2.0
    generated_images = np.clip(generated_images, 0, 1)
    
    # 可视化
    plt.figure(figsize=(20, 2))
    for i in range(n_steps):
        plt.subplot(1, n_steps, i + 1)
        plt.imshow(generated_images[i])
        plt.axis('off')
        if i == 0:
            plt.title('起点', fontsize=10)
        elif i == n_steps - 1:
            plt.title('终点', fontsize=10)
    
    plt.tight_layout()
    plt.savefig(f'{output_dir}/latent_interpolation.png', dpi=150, bbox_inches='tight')
    plt.show()

# 执行插值实验
print('潜在空间插值实验：')
interpolate_latent_space(generator, latent_dim, n_steps=10)
print(f'插值结果已保存到: {output_dir}/latent_interpolation.png')

# ============================================================================
# 批量生成图像
# ============================================================================

def generate_image_grid(generator, latent_dim, rows=5, cols=10):
    """
    批量生成图像网格
    
    参数:
        generator: 生成器模型
        latent_dim: 潜在空间维度
        rows: 行数
        cols: 列数
    """
    n_images = rows * cols
    noise = np.random.normal(0, 1, size=(n_images, latent_dim))
    generated_images = generator.predict(noise, verbose=0)
    generated_images = (generated_images + 1) / 2.0
    generated_images = np.clip(generated_images, 0, 1)
    
    plt.figure(figsize=(cols * 2, rows * 2))
    for i in range(n_images):
        plt.subplot(rows, cols, i + 1)
        plt.imshow(generated_images[i])
        plt.axis('off')
    
    plt.tight_layout()
    plt.savefig(f'{output_dir}/generated_grid.png', dpi=150, bbox_inches='tight')
    plt.show()

print('\n生成图像网格：')
generate_image_grid(generator, latent_dim, rows=5, cols=10)
print(f'图像网格已保存到: {output_dir}/generated_grid.png')